<a href="https://colab.research.google.com/github/wtglad/jhu-coursera-making-decisions-in-time/blob/main/%20information-theory-and-games/Diplo_Parsing_Payoffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [98]:
from ndicts import DataDict, NestedDict

In [47]:
from progressbar import ProgressBar
import math

In [2]:
folders = []

for i in os.listdir('/content/drive/MyDrive/Diplomacy/game-states-batch1/'):
  folders.append(['/content/drive/MyDrive/Diplomacy/game-states-batch1/', i])

In [7]:
files = []
pbar = ProgressBar()

for folder in pbar(folders):
  for file in os.listdir(folder[0] + folder[1]):
    if 'payoff' in file:
      files.append([folder[0], folder[1], file])

100% (3829 of 3829) |####################| Elapsed Time: 0:03:49 Time:  0:03:49


In [10]:
file_df = pd.DataFrame(files, columns = ['parent_dir', 'game_id', 'file'])

In [11]:
file_df

,parent_dir,game_id,file
0,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,20 - S1906M - alliance_attacker_payoffs.csv
1,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,21 - F1906M - alliance_attacker_payoffs.csv
2,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,1 - S1901M - alliance_attacker_payoffs.csv
3,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,2 - F1901M - alliance_attacker_payoffs.csv
4,/content/drive/MyDrive/Diplomacy/game-states-b...,111483,4 - S1902M - alliance_attacker_payoffs.csv
...,...,...,...
65329,/content/drive/MyDrive/Diplomacy/game-states-b...,112901,52 - S1914M - alliance_attacker_payoffs.csv
65330,/content/drive/MyDrive/Diplomacy/game-states-b...,112901,53 - F1914M - alliance_attacker_payoffs.csv
65331,/content/drive/MyDrive/Diplomacy/game-states-b...,112901,54 - S1915M - alliance_attacker_payoffs.csv
65332,/content/drive/MyDrive/Diplomacy/game-states-b...,112901,55 - F1915M - alliance_attacker_payoffs.csv


In [12]:
game_id = file_df['game_id'].iloc[0]

In [13]:
game_id

'111483'

In [17]:
game_files = file_df[file_df['game_id'] == game_id].reset_index(drop=True)

In [110]:
payoff_dict = {}
pbar = ProgressBar()

for game_id in pbar(file_df['game_id'].unique()):
  
  game_files = file_df[file_df['game_id'] == game_id].reset_index(drop=True)

  for i in game_files.index:
      row = game_files.iloc[i]
      phase = row['file'].split('-')[1].strip()
      phase_payoffs = pd.read_csv(row['parent_dir'] + game_id + '/' + row['file'])

      phase_payoffs = phase_payoffs[phase_payoffs['power'].notnull()].reset_index(drop=True)

      for power in phase_payoffs['power'].to_list():
        
        if power not in payoff_dict.keys():
          payoff_dict[power] = {}

        if phase not in payoff_dict[power].keys():
          payoff_dict[power][phase] = {}

        cooperates = phase_payoffs[phase_payoffs['power'] == power]['cooperates'].iloc[0]
        if pd.isnull(cooperates):
          cooperates = frozenset()
        if len(cooperates) > 0:
          cooperates = frozenset(set(eval(cooperates)))

        attacks = phase_payoffs[phase_payoffs['power'] == power]['attacks'].iloc[0]
        if pd.isnull(attacks):
          attacks = frozenset()
        if len(attacks) > 0:
          attacks = frozenset(set(eval(attacks)))

        if (cooperates, attacks) not in payoff_dict[power][phase].keys():
          payoff_dict[power][phase][(cooperates, attacks)] = []

        payoff = phase_payoffs[phase_payoffs['power'] == power]['payoff'].iloc[0]
        payoff_dict[power][phase][(cooperates, attacks)].append(payoff)

100% (3810 of 3810) |####################| Elapsed Time: 3:13:36 Time:  3:13:36


In [111]:
import pickle 

with open('/content/drive/MyDrive/Diplomacy/game-states-batch1-payoff_dictionary.pkl', 'wb') as f:
    pickle.dump(payoff_dict, f)